# Designing a pipeline of the project

In [1]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [2]:
import pandas as pd
import pickle

In [3]:
test=pd.read_csv("/content/drive/My Drive/mercedes_testdata.csv")

#### picking the random point from the test dataset

In [4]:
#picking Random point from the test data.
import random
rand=random.choice(test.values)
# Let construct the dataset.
query_point=pd.DataFrame(data=rand.reshape(1,-1),columns=test.columns)



#### Random query data point from the test dataset provided by the mercedes benz

In [5]:
query_point

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,...,X345,X346,X347,X348,X349,X350,X351,X352,X353,X354,X355,X356,X357,X358,X359,X360,X361,X362,X363,X364,X365,X366,X367,X368,X369,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,5503,n,v,ae,c,d,m,a,w,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [6]:
trainX0=pickle.load(open("/content/drive/My Drive/meanX0","rb"))
trainX1=pickle.load(open("/content/drive/My Drive/meanX1","rb"))
trainX2=pickle.load(open("/content/drive/My Drive/meanX2","rb"))
trainX3=pickle.load(open("/content/drive/My Drive/meanX3","rb"))
trainX4=pickle.load(open("/content/drive/My Drive/meanX4","rb"))
trainX5=pickle.load(open("/content/drive/My Drive/meanX5","rb"))
trainX6=pickle.load(open("/content/drive/My Drive/meanX6","rb"))
trainX8=pickle.load(open("/content/drive/My Drive/meanX8","rb"))


#### performing preprocess on query point

In [7]:
def preprocess(X):
  # checking The null values
  if X.isnull().values.any()==True:
    return X.fillna(0)
  else:
    return X

In [8]:
df=preprocess(query_point)
df

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,...,X345,X346,X347,X348,X349,X350,X351,X352,X353,X354,X355,X356,X357,X358,X359,X360,X361,X362,X363,X364,X365,X366,X367,X368,X369,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,5503,n,v,ae,c,d,m,a,w,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [9]:
df.shape

(1, 377)

#### performing Feature Engineering For a query point

---



In [10]:
def feature_engineering(X):

  # Making Mean Encoding for categorical variables
  y_train=pickle.load(open("/content/drive/My Drive/ground_truth","rb"))
  y_mean=y_train.mean()
  X["X0"]=[trainX0.loc[X["X0"].iloc[0]] if X["X0"].iloc[0] in trainX0.index else y_mean ]
  X["X1"]=[trainX1.loc[X["X1"].iloc[0]] if X["X1"].iloc[0] in trainX1.index else y_mean ]
  X["X2"]=[trainX2.loc[X["X2"].iloc[0]] if X["X2"].iloc[0] in trainX2.index else y_mean ]
  X["X3"]=[trainX3.loc[X["X3"].iloc[0]] if X["X3"].iloc[0] in trainX3.index else y_mean ]
  X["X4"]=[trainX4.loc[X["X4"].iloc[0]] if X["X4"].iloc[0] in trainX4.index else y_mean ]
  X["X5"]=[trainX5.loc[X["X5"].iloc[0]] if X["X5"].iloc[0] in trainX5.index else y_mean ]
  X["X6"]=[trainX6.loc[X["X6"].iloc[0]] if X["X6"].iloc[0] in trainX6.index else y_mean ]
  X["X8"]=[trainX8.loc[X["X8"].iloc[0]] if X["X8"].iloc[0] in trainX8.index else y_mean ]
  
  # Adding Interactive Features

  X["X136_add_X261"]=X.apply(lambda row: row.X136 + row.X261, axis=1)
  X["X118_add_X275"]=X.apply(lambda row: row.X118 + row.X275, axis=1)
  X["X314_add_X315"]=X.apply(lambda row: row.X314 + row.X315,axis=1)
  X["X0_add_X1"]=X.apply(lambda row: row.X0 + row.X1,axis=1)
  X["X263_add_X264"]=X.apply(lambda row: row.X263 + row.X264,axis=1) 
  X["X136_add_X275_add_X261"]=X.apply(lambda row: row.X136 + row.X275 + row.X261, axis=1)
  X["X118_add_X314_add_X315"]=X.apply(lambda row: row.X118 + row.X314 + row.X315, axis=1)
  X["X261_add_X263_add_X264"]=X.apply(lambda row: row.X261 + row.X263 + row.X264, axis=1)
  X["X0_add_X1_add_X2"]=X.apply(lambda row: row.X0 + row.X1 + row.X2, axis=1)
  
  # Removing Zero variance features,low variance,duplicate features and highly correlated features
  removal_features=pickle.load(open("/content/drive/My Drive/rmoval_feat.pkl","rb"))
  X=X.drop(removal_features,axis=1)

  # selecting the best features using SelectKbest
  best_feat=pickle.load(open("/content/drive/My Drive/best_columns.pkl","rb"))
  for i in X.columns:
    if i not in best_feat:
      X=X.drop(i,axis=1)

  return X

In [11]:
df=feature_engineering(query_point)
df

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X12,X13,X14,X19,X20,X22,X23,X27,X28,X29,X31,X43,X44,X45,X46,X47,X48,X50,X51,X52,X56,X61,X64,X66,X68,X69,X71,X75,X80,X81,X82,...,X275,X276,X283,X284,X285,X286,X291,X300,X301,X304,X305,X306,X313,X315,X316,X329,X331,X334,X336,X337,X342,X343,X346,X349,X350,X351,X354,X355,X373,X376,X377,X136_add_X261,X118_add_X275,X314_add_X315,X0_add_X1,X263_add_X264,X136_add_X275_add_X261,X118_add_X314_add_X315,X261_add_X263_add_X264,X0_add_X1_add_X2
0,5503,93.491333,100.983086,97.103225,101.623027,100.349008,101.243058,97.802524,98.325959,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,1,0,...,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,194.47442,1,1,0,1,291.577645


In [12]:
df.shape

(1, 146)

#### Predicting the query point with best model of stack model

In [13]:
def predict(X):
  a=preprocess(X)
  a=feature_engineering(a)
  stack_saved = pickle.load(open("/content/drive/My Drive/finalized_model.pkl","rb"))
  predict_value=stack_saved.predict(a.values)
  return predict_value

#### Result of the predicted value

In [14]:
predicted=predict(query_point)

[03:55:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:55:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [15]:
print("The predicted value is:",predicted[0],"seconds")

The predicted value is: 97.51994746054754 seconds


### Therefore the given input query data point the above is the predicted output